In [85]:
import pandas as pd
import nltk
from pythainlp.corpus import stopwords
from pythainlp.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tag import CRFTagger

In [86]:
from sklearn.multiclass import OneVsRestClassifier
#Load the dataset
#df = pd.read_csv(f'w_review_train.csv', sep=';', header=None, names=['review', 'star'])
df = pd.read_csv(f'wongnai_reviews2k.csv')

In [87]:
df.head()

,reviews,star
0,ไม่รู้จะใจร้ายไปรึเปล่า แต่ไม่อร่อยอ่ะค่ะ\r\n\...,0
1,ขอออกความเห็นแบบตรงไปตรงมานะครับ...\r\n\r\nผมไ...,0
2,วันนี้ไปทานมา. chocolateหวานเลี่ยนเกินไปค่ะ ตั...,0
3,สวัสดีค่ะ เพื่อนๆชาวนักกินทุกคน^^\r\nไม่ได้เจอ...,0
4,รีวิวในครั้งนี้ไม่ใช่สเต็กในร้าน steak lao นะค...,0


In [88]:
df.shape

(2000, 2)

In [89]:
def word_stop(token_list):
    stop_words = set(stopwords.words("thai"))
    stopword2 = [',','-',"'",'"','_','(',')',':','?','??','???','????','?????','???????','????????','?????????','?????????','????????????????????','%','ก','The','ต์','แล้','..','ด','ๆดึ๊กๆ','฿)','฿', '^^','^','^^^','ว','ๆๆๆๆๆๆๆๆๆ','ร่????','าา????????????????','เ','ๆๆ????????????????','ๆ','ๆๆ','ๆๆๆ','ๆๆๆๆ','ๆๆๆๆๆ','ๆๆๆๆๆๆ','ๆๆๆๆๆๆๆ','ๆๆๆๆๆๆๆๆ','ๆ!!!','ๆ(','ยยยยยยยยย','ท','<','>',' ','  ','   ','    ','     ','      ','       ','        ','                \t','ล','ง','กก','น','ส','บอ','รี','ร์','ริ','ฟ','ก้อ','เส','ชาด','วๆ','ออ','บ','ป','รา','อ่ะ','สี','ค','เด','กา','ต','์','อะ','บิ','ฟิ','ม','บุ','พน','กิ','T','ย','รี่','ง.','B','วิ','อ','า','ร','ฮุฮิ','ฟ่าาา','รื่อยๆ','ไห','าาาา','555','the','>_________________<','                     ','                                                                                                                              ','               ','>_________________<','^^?','ยยยยยยยยยยยยยยยยยยยยยย','ๆ????','มม????','ไห','ยยยยย','าาาา','วดี','ๆๆ!!!','ยย','ย่','~*','^_____^*','^___^','+','++','><','าา','<<','จิ','ระ','วว','ช','1.','2.','4.','5.','1','2','3','4','5','6','7','8','9','10','3.','.','วจะ','รึ','ร้่','^_^','ท๊','ยยย','sq','ล์ๆ','น.','/','!!!','มๆ','=','!!','คๆ','           ','ี๊่ยว','้','ดๆ','ฟเฟ่','เบๆ','ฯ', 'บูๆ','รุ้','เด้','!','\t','ยุ่','่','”','น่','ปต่','^_^','d','าาา','::','็','ะ','฿):','.ๆ','าเเถวๆ','กัจะ','ดดด','กี๊','):','ห','ก้','น้','*','ถ.','น้ล็กๆ','ร้','ยๆ','ป๊','\r\n','……????','…','•~•''','๕๕๕๕','๕๕','๑๒๐','ํป','์เล่','์เล','์เปปเป','์เค','็ะ','ๆๆ.','ๆเฟ่','ๆเชงๆ','ๆว้','ๆฟิ','ๆผ','ๆกัร','ๆ ','ๆ.....','ๆ...','ๆ"','ๆ','ไ้ด้','ไื','ไา้','ไฮเวย์','ไมา','ไก้','ใี','ใา','ให','ใทั้ง','โอกะ','แ้','แ่ซ่','แค้','แข','เ้วย','เ็ป็','เ็','เสิร์','เว่่','เลที่ยะ','เลก็พื้','เลก็ดี','เร้ยยย!!!!','เรืี่','เรี','เม.ย.','เฟ่ต์','เฟิม','เปี่','เปิง','เปลี้ย','เปลี่ย','เปร้ยว','เปรั้ยว','เปรยๆ','เปร','เปปเป','เบล','เบย)','เบยย','เบยยยย','เนล"','เดิฟท่ี','เซ็ต"','เชอ','เชฟเฟ','เฉ','เงิบ!!!','฿.','฿+++','ู','ุ','ื่ม','ืำ','ืา','ึะ','ี้','ีว','าาาาาาาา','าาาาาา','าม','า.','ัว','ั','ะะะะะ????????????????????????','ฮ๊','ฮ้ง','ฮี่ฮี่ๆๆ','ฮั้ง','ฮั่ง','ฮัทตั้น','ฮัท','ฮัง','ฮง','อ๊','อื้ม..!!','อือออ','อึ๊งๆ','อี๊','อิ๊ญ','ห์','ห้่','หุหุ','หุ)','หิม','หาว่า','หว๊า','หว','หลืบ','หลาม','หรัล','หมาดๆ','หป','ส้อม','สึ','สะ่ง','สส','สวท.','สต.','ษฯ','ศ์','ศุ','ศศ','ศก.','ศ.','ว๊า','ว้','ว่่่่า','วืช','วล์ๆ','วม','ล์ค(','ล์ค','ล่ง','ล่','ลู่','ลุ','ลู','ลื้มมมมมมม','ลล์"','ล/ค','ล.','ล"','ร์เร','ร์เนีย','ร์เดิร์ฟอิ','ร์อิ','ร์)','ร๋ๆ','ร๊า','ร้แย','ร่','~_~',' (','~_~','yukke','yogurt','________','__','^^~','^^)','^^"','^.^)','^....^','@','?ๆ','????????????????????????','??????????','????????)','????:','????)','???(','??)','?...(','?..','?(','?!','>_<','>\<','><~','><!','>','<',';;',';))',';(',';"(',':)))',':(','990','98.947775','931087','90.','9.00','9.','899','8594','8549','8534943','850','840','820','8077','80.','8.','79','78','77','756','7131','71','699','698','69.','670','6655022','665','653','650','65.','640','63','620','6157254','6.5','6.00','599','5796','57','555555555','55555555','555..','5500','550.','545','540','54','539','51','50.','489','48','470','465','460','450.','450','449','429','400.','4.6','4.5..','395','394','389','379','375','370','37','364','37','364','360','36','357','3569','355','35.','334','3200','320','32','31','309','304','3.7','3.4','3.30','3.1','3..','2900','289','28','270','27','2650','265','26','2555','25.','248','241','24.00','234','229','225','224','22.30','219','209','205','2017','2016','2003','2000','2.5..','2.5','194','1927','190','19.372471','19.30','188','185.','180.','18.99','18.30','17.5','165.','164','1600','16.99','159','150.','147','145.','145','14.30','14.00','135','1300','13.00','120.','12.','118','11.45','11.30','1090','1080','108','100.','10..','1.30','1.3..','1.2..','096','0957235785','091','09.00','089','089','084','083','0819591628','080','07.00','05','03.30','02','000.','0.5','..วรัญ','..????','...อื่มมม','....^^','........','.......','-nutty','-Toast','-Special','-Roll','-Pudding','-Lava','-Icecream','-Half','-Eleven','-Breadstick','+คื่น','++)','*******','))',')"','(ใส้','(เอ๋...','(เลิฟๆ','(มั้ง?)','(ปิง','(นำ้','%***','%)','"เฟ','"เบ','"เชล','"ฮัน','"อู๋','"..','".','!"','!!!????????????????????','!!!?','!!!)','!!!!^^*)','','!!!!!!','!!!','                                                           ','                                 ','                          ','                 ','                 ','••','’','ๆ..','5555', '19', '30','25','scoop']
    stopword3 = [''];
    filtered_sentence = []
    for w in token_list:
        #if w not in stop_words and w not in stopword2:
        if w not in stopword2:
        #if w not in stopword3:
            filtered_sentence.append(w)
    return filtered_sentence

In [90]:
documents = []
for i in range(0, len(df)):
    all_words = []
    words = word_tokenize(df['reviews'][i])
    words = word_stop(words)
    for w in words:
        all_words.append(w)
    documents.append(all_words)

In [91]:
# documents = []

# allowed_word_types = ["A","V","X"]
# ct = CRFTagger()
# ct.set_model_file('model.crf.tagger')

# for i in range(0, len(df)):
#     all_words = []
#     words = word_tokenize(df['reviews'][i])
#     words = word_stop(words)
#     pos = ct.tag_sents([words])

#     for w in pos[0]:
#         if w[1][0] in allowed_word_types:
#             all_words.append(w[0])
#     documents.append(all_words)

In [92]:
print(documents[0])

['ไม่รู้', 'จะ', 'ใจร้าย', 'ไป', 'รึเปล่า', 'แต่', 'ไม่อร่อย', 'ค่ะ', 'Chocolate', 'ที่', 'ให้', 'มา', 'สำหรับ', 'จิ้ม', 'เย็น', 'ได้', 'โล่', 'ไม่มี', 'ไฟ', 'อัง', 'มา', 'ให้', 'ด้วย', 'ค่ะ', 'ใส่', 'ถ้วย', 'มา', 'เฉย', 'โฆษณา', 'ว่า', 'เป็น', 'chocolate', 'มาจาก', 'Swiss', 'แต่', 'หวาน', 'มาก', 'ประมาณ', 'ว่า', 'เอา', 'อะไร', 'ไป', 'จิ้ม', 'ไม่ได้', 'เลย', 'เครื่อง', 'ที่', 'ใช้', 'จิ้ม', 'เห่', 'มาก', 'Waffle', 'แข็ง', 'กรอบ', 'ส่วน', 'ราว', 'นี่', 'ก็', 'หวาน', 'สุด', 'บวก', 'กับ', 'chocolate', 'ที่', 'หวาน', 'อยู่', 'แล้ว', 'แทบตาย', 'เลย', 'ทีเดียว', 'ไป', 'ลอง', 'กัน', 'ได้', 'ค่ะ', 'ลักษณะ', 'ร้าน', 'เป็น', 'ร้าน', 'คาเฟ่', 'เล็ก', 'ริม', 'ทางใน', 'ห้าง', 'มี', 'เก้าอี้นั่ง', 'ได้', 'ประมาณ', 'ที่', 'ค่ะ', 'ราคาขาย', 'ไม่', 'แพง', 'มาก', 'สำหรับ', 'ฟอง', 'ดู', 'แต่', 'ก็', 'ไม่ได้', 'ถูก', 'ค่ะ']


In [93]:
document_tfidf = [" ".join(review) for review in documents]

In [94]:
print(document_tfidf[0])

ไม่รู้ จะ ใจร้าย ไป รึเปล่า แต่ ไม่อร่อย ค่ะ Chocolate ที่ ให้ มา สำหรับ จิ้ม เย็น ได้ โล่ ไม่มี ไฟ อัง มา ให้ ด้วย ค่ะ ใส่ ถ้วย มา เฉย โฆษณา ว่า เป็น chocolate มาจาก Swiss แต่ หวาน มาก ประมาณ ว่า เอา อะไร ไป จิ้ม ไม่ได้ เลย เครื่อง ที่ ใช้ จิ้ม เห่ มาก Waffle แข็ง กรอบ ส่วน ราว นี่ ก็ หวาน สุด บวก กับ chocolate ที่ หวาน อยู่ แล้ว แทบตาย เลย ทีเดียว ไป ลอง กัน ได้ ค่ะ ลักษณะ ร้าน เป็น ร้าน คาเฟ่ เล็ก ริม ทางใน ห้าง มี เก้าอี้นั่ง ได้ ประมาณ ที่ ค่ะ ราคาขาย ไม่ แพง มาก สำหรับ ฟอง ดู แต่ ก็ ไม่ได้ ถูก ค่ะ


In [95]:
df['review_cut'] = document_tfidf
df.head()

,reviews,star,review_cut
0,ไม่รู้จะใจร้ายไปรึเปล่า แต่ไม่อร่อยอ่ะค่ะ\r\n\...,0,ไม่รู้ จะ ใจร้าย ไป รึเปล่า แต่ ไม่อร่อย ค่ะ C...
1,ขอออกความเห็นแบบตรงไปตรงมานะครับ...\r\n\r\nผมไ...,0,ขอ ออกความเห็น แบบ ตรงไปตรงมา นะ ครับ ... ผม ไ...
2,วันนี้ไปทานมา. chocolateหวานเลี่ยนเกินไปค่ะ ตั...,0,วันนี้ ไป ทาน มา chocolate หวาน เลี่ยน เกินไป ...
3,สวัสดีค่ะ เพื่อนๆชาวนักกินทุกคน^^\r\nไม่ได้เจอ...,0,สวัสดี ค่ะ เพื่อน ชาว นักกิน ทุกคน ไม่ได้ เจอก...
4,รีวิวในครั้งนี้ไม่ใช่สเต็กในร้าน steak lao นะค...,0,วิว ใน ครั้งนี้ ไม่ใช่ สเต็ก ใน ร้าน steak lao...


In [96]:
df['text length'] = df['review_cut'].apply(len)
df.head()

,reviews,star,review_cut,text length
0,ไม่รู้จะใจร้ายไปรึเปล่า แต่ไม่อร่อยอ่ะค่ะ\r\n\...,0,ไม่รู้ จะ ใจร้าย ไป รึเปล่า แต่ ไม่อร่อย ค่ะ C...,508
1,ขอออกความเห็นแบบตรงไปตรงมานะครับ...\r\n\r\nผมไ...,0,ขอ ออกความเห็น แบบ ตรงไปตรงมา นะ ครับ ... ผม ไ...,391
2,วันนี้ไปทานมา. chocolateหวานเลี่ยนเกินไปค่ะ ตั...,0,วันนี้ ไป ทาน มา chocolate หวาน เลี่ยน เกินไป ...,239
3,สวัสดีค่ะ เพื่อนๆชาวนักกินทุกคน^^\r\nไม่ได้เจอ...,0,สวัสดี ค่ะ เพื่อน ชาว นักกิน ทุกคน ไม่ได้ เจอก...,1824
4,รีวิวในครั้งนี้ไม่ใช่สเต็กในร้าน steak lao นะค...,0,วิว ใน ครั้งนี้ ไม่ใช่ สเต็ก ใน ร้าน steak lao...,637


In [97]:
X = df['review_cut']
y = df['star']

In [98]:
X[0]

'ไม่รู้ จะ ใจร้าย ไป รึเปล่า แต่ ไม่อร่อย ค่ะ Chocolate ที่ ให้ มา สำหรับ จิ้ม เย็น ได้ โล่ ไม่มี ไฟ อัง มา ให้ ด้วย ค่ะ ใส่ ถ้วย มา เฉย โฆษณา ว่า เป็น chocolate มาจาก Swiss แต่ หวาน มาก ประมาณ ว่า เอา อะไร ไป จิ้ม ไม่ได้ เลย เครื่อง ที่ ใช้ จิ้ม เห่ มาก Waffle แข็ง กรอบ ส่วน ราว นี่ ก็ หวาน สุด บวก กับ chocolate ที่ หวาน อยู่ แล้ว แทบตาย เลย ทีเดียว ไป ลอง กัน ได้ ค่ะ ลักษณะ ร้าน เป็น ร้าน คาเฟ่ เล็ก ริม ทางใน ห้าง มี เก้าอี้นั่ง ได้ ประมาณ ที่ ค่ะ ราคาขาย ไม่ แพง มาก สำหรับ ฟอง ดู แต่ ก็ ไม่ได้ ถูก ค่ะ'

In [99]:
y[0]

0

In [100]:
count_vect = TfidfVectorizer(lowercase=True,min_df=2).fit(df['review_cut'])

In [101]:
X_count_vect = count_vect.transform(df['review_cut'])

In [102]:
X_names = count_vect.get_feature_names()
X_names

['00',
 '000',
 '02',
 '05',
 '053',
 '07',
 '081',
 '09',
 '10',
 '100',
 '1000',
 '101',
 '105',
 '109',
 '11',
 '110',
 '1112',
 '1122',
 '115',
 '12',
 '120',
 '1200',
 '125',
 '129',
 '13',
 '130',
 '139',
 '14',
 '140',
 '15',
 '150',
 '1500',
 '155',
 '16',
 '160',
 '165',
 '169',
 '17',
 '170',
 '175',
 '179',
 '18',
 '180',
 '185',
 '189',
 '19',
 '195',
 '199',
 '20',
 '200',
 '2008',
 '2012',
 '2013',
 '2014',
 '2017',
 '21',
 '210',
 '22',
 '220',
 '23',
 '230',
 '239',
 '24',
 '240',
 '249',
 '250',
 '2557',
 '2560',
 '259',
 '260',
 '27',
 '280',
 '29',
 '290',
 '295',
 '299',
 '30',
 '300',
 '315',
 '325715429710',
 '34',
 '340',
 '349',
 '35',
 '350',
 '351',
 '38',
 '380',
 '385',
 '39',
 '390',
 '399',
 '40',
 '400',
 '42',
 '420',
 '43',
 '430',
 '45',
 '46',
 '47',
 '479',
 '480',
 '49',
 '499',
 '50',
 '500',
 '53',
 '530',
 '55',
 '55555',
 '555555',
 '5555555',
 '56',
 '58',
 '59',
 '60',
 '600',
 '65',
 '69',
 '690',
 '70',
 '700',
 '75',
 '750',
 '760',
 '80',


In [103]:
X_count_vect = pd.DataFrame(X_count_vect.toarray(),columns=X_names)

In [104]:
X_count_vect.shape

(2000, 4244)

In [105]:
X_count_vect.head()

,00,000,02,05,053,07,081,09,10,100,...,ไอต,ไอศกร,ไอศคร,ไอเด,ไฮ,ไฮโซ,ๆก,ๆๆ,ๆๆๆ,๕๕๕
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [119]:
X_train_cv , X_test_cv , y_train_cv , y_test_cv = train_test_split(X_count_vect,y,test_size=0.10,random_state=2499)

In [120]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_cv, y_train_cv)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [121]:
preds = nb.predict(X_test_cv)

In [122]:
from sklearn.metrics import confusion_matrix, classification_report , accuracy_score
print(confusion_matrix(y_test_cv, preds))
print('\n')
print(classification_report(y_test_cv, preds))
print("accuracy_score" , accuracy_score(y_test_cv, preds))
#print("roc_auc_score" , roc_auc_score(y_test , mnb.predict_proba(X_test)[:,1]))

[[87 18]
 [ 4 91]]


             precision    recall  f1-score   support

          0       0.96      0.83      0.89       105
          1       0.83      0.96      0.89        95

avg / total       0.90      0.89      0.89       200

accuracy_score 0.89


In [123]:
from sklearn.model_selection import KFold

In [124]:
kf = KFold(n_splits=10, random_state=2499, shuffle=True)
accuracy_scores = []
sum_average = 0
for X_train_cv , X_test_cv in kf.split(X_count_vect):
    print("train = ", len(X_train_cv)," test = ", len(X_test_cv))
    train_data = np.array(X_count_vect)[X_train_cv]
    test_data = np.array(X_count_vect)[X_test_cv]
    y_train_cv = np.array(y)[X_train_cv]
    y_test_cv =  np.array(y)[X_test_cv]
    clf_cv = MultinomialNB()
    clf_cv.fit(train_data,y_train_cv)
    y_pred_cv = clf_cv.predict(test_data)
    print(confusion_matrix(y_test_cv, y_pred_cv))
    print('\n')
    print(classification_report(y_test_cv, y_pred_cv))
    print("accuracy_score" , accuracy_score(y_test_cv, y_pred_cv))
    print("\n")
    sum_average += accuracy_score(y_test_cv, y_pred_cv)
#print("roc_auc_score" , roc_auc_score(y_test , nb.predict_proba(X_test)[:,1]))
average = (sum_average/10)*100 
print("Naive Bayes Accuracy average >> " , average)
print('===============================================\n')

train =  1800  test =  200
[[87 18]
 [ 4 91]]


             precision    recall  f1-score   support

          0       0.96      0.83      0.89       105
          1       0.83      0.96      0.89        95

avg / total       0.90      0.89      0.89       200

accuracy_score 0.89


train =  1800  test =  200
[[82 19]
 [ 2 97]]


             precision    recall  f1-score   support

          0       0.98      0.81      0.89       101
          1       0.84      0.98      0.90        99

avg / total       0.91      0.90      0.89       200

accuracy_score 0.895


train =  1800  test =  200
[[76 23]
 [ 8 93]]


             precision    recall  f1-score   support

          0       0.90      0.77      0.83        99
          1       0.80      0.92      0.86       101

avg / total       0.85      0.84      0.84       200

accuracy_score 0.845


train =  1800  test =  200
[[85 20]
 [ 4 91]]


             precision    recall  f1-score   support

          0       0.96      0.81      0.8

# Test Preditction

In [252]:
df_test = pd.read_csv(f'food_sent_raw.csv')

In [253]:
df_test.head()

,src,target,text
0,w,1,ร้านนี้ฮิตมานานในหมู่นักศึกษา มช.ค่ะ (แน่หละ ก...
1,w,1,ร้านนี้อาจจะต้องบริการตนเองนิดนึงนะคะ เพราะคนเ...
2,w,1,ร้านนี้อาหารอร่อยทุกอย่างไม่ว่าจะเป็นส้มตำไก่ท...
3,w,1,ร้านขนมจีนเส้นสด พร้อมอาหารตามสั่งที่นี่ จะเสร...
4,w,1,ร้านก๋วยจั๊บเปิดใหม่ใกล้ ๆ ข่วงสันกำแพง ทางไปว...


In [254]:
len(df_test)

2000

In [255]:
documents_test = []
for i in range(0, len(df_test)):
    all_words = []
    words = word_tokenize(df_test['text'][i])
    words = word_stop(words)
    for w in words:
        all_words.append(w)
    documents_test.append(all_words)

In [256]:
len(documents_test)

2000

In [257]:
document_tfidf_test = [" ".join(review) for review in documents_test]

In [258]:
len(document_tfidf_test)

2000

In [259]:
df_test['text_cut'] = document_tfidf_test
df_test.head()

,src,target,text,text_cut
0,w,1,ร้านนี้ฮิตมานานในหมู่นักศึกษา มช.ค่ะ (แน่หละ ก...,ร้าน นี้ ฮิต หมู่ นักศึกษา มช. แน่ หละ มอ นิ ช...
1,w,1,ร้านนี้อาจจะต้องบริการตนเองนิดนึงนะคะ เพราะคนเ...,ร้าน นี้ บริการ นิดนึง นะคะ คน ร้าน อาหาร อร่อ...
2,w,1,ร้านนี้อาหารอร่อยทุกอย่างไม่ว่าจะเป็นส้มตำไก่ท...,ร้าน นี้ อาหาร อร่อย ส้มตำ ไก่ทอด สมุนไพร ขนมจ...
3,w,1,ร้านขนมจีนเส้นสด พร้อมอาหารตามสั่งที่นี่ จะเสร...,ร้าน ขนมจีน เส้น สด อาหาร สั่ง ที่นี่ ขนมจีน ต...
4,w,1,ร้านก๋วยจั๊บเปิดใหม่ใกล้ ๆ ข่วงสันกำแพง ทางไปว...,ร้าน ก๋วยจั๊บ ข่วง สัน กำแพง วัด สัน เหนือ ตึก...


In [263]:
df_test['target'][1200]

1

In [272]:
positive_review = "ร้านสกปรก ไม่สะอาดเลย"
print(positive_review)
positive_review_transformed = bow_transformer.transform([positive_review])
nb.predict(positive_review_transformed)[0]

ร้านสกปรก ไม่สะอาดเลย


0